In [106]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from scipy.stats import mode
from tqdm.notebook import tqdm
from p_tqdm import p_map
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_combined = pd.read_csv('../../search_results/LOPQ+Both+resnet18+256+20.csv', index_col=None)
excluded_classes = ['0235_D01_S001', '0200_D01_S001', '0049_D01_S001', '0842_D01_S001', '0337_D01_S001', '0263_D01_S001', '0735_D01_S001', '0541_D01_S001', '0311_D01_S001']
len(excluded_classes)
df_combined = df_combined[~df_combined.package_actual.isin(excluded_classes)]
df_combined.package_actual.nunique()

989

In [4]:
df_bad_samples = pd.read_csv('to_investigate.csv', index_col=None)

In [9]:
df_selected = df_combined[df_combined.sample_actual.isin(df_bad_samples['sample_actual'].to_list())]

In [16]:
dfs = [x for _, x in df_selected.groupby('sample_actual')]

In [87]:
df.columns

Index(['distance', 'uuid_actual', 'uuid_predicted', 'sample_actual',
       'package_actual', 'descriptor_actual', 'package_predicted', 'algorithm',
       'match'],
      dtype='object')

In [98]:
df = dfs[0]

sample_actual = df.iloc[0].sample_actual
package_actual = df.iloc[0].package_actual
alg = df.algorithm.unique()[0]
df[df.algorithm == 'MI1']

,distance,uuid_actual,uuid_predicted,sample_actual,package_actual,descriptor_actual,package_predicted,algorithm,match
25702240,4.277310,PFP_Ph1_P0031_D01_S001_C2_az080_side1_0005,PFP_Ph1_P0031_D01_S001_C2_az360_side1_0008,Ph1_P0031_D01_S001_C2_az080,0031_D01_S001,5,0031_D01_S001,MI1,True
25702241,6.858270,PFP_Ph1_P0031_D01_S001_C2_az080_side1_0005,PFP_Ph1_P0667_D01_S001_C2_az360_side1_0010,Ph1_P0031_D01_S001_C2_az080,0031_D01_S001,5,0667_D01_S001,MI1,False
25702242,7.412381,PFP_Ph1_P0031_D01_S001_C2_az080_side1_0005,PFP_Ph1_P0895_D01_S001_C3_az360_side1_0036,Ph1_P0031_D01_S001_C2_az080,0031_D01_S001,5,0895_D01_S001,MI1,False
25702243,7.684181,PFP_Ph1_P0031_D01_S001_C2_az080_side1_0005,PFP_Ph3_P0031_D01_S001_C2_az360_side1_0005,Ph1_P0031_D01_S001_C2_az080,0031_D01_S001,5,0031_D01_S001,MI1,True
25702244,7.737307,PFP_Ph1_P0031_D01_S001_C2_az080_side1_0005,PFP_Ph1_P0685_D01_S001_C4_az360_side1_0015,Ph1_P0031_D01_S001_C2_az080,0031_D01_S001,5,0685_D01_S001,MI1,False
...,...,...,...,...,...,...,...,...,...
33637755,8.099594,PFP_Ph1_P0031_D01_S001_C2_az080_side1_0003,PFP_Ph1_P0150_D01_S001_C3_az360_side1_0003,Ph1_P0031_D01_S001_C2_az080,0031_D01_S001,3,0150_D01_S001,MI1,False
33637756,8.191508,PFP_Ph1_P0031_D01_S001_C2_az080_side1_0003,PFP_Ph3_P0211_D01_S001_C1_az360_side1_0001,Ph1_P0031_D01_S001_C2_az080,0031_D01_S001,3,0211_D01_S001,MI1,False
33637757,8.249652,PFP_Ph1_P0031_D01_S001_C2_az080_side1_0003,PFP_Ph3_P0631_D01_S001_C2_az360_side1_0009,Ph1_P0031_D01_S001_C2_az080,0031_D01_S001,3,0631_D01_S001,MI1,False
33637758,8.447738,PFP_Ph1_P0031_D01_S001_C2_az080_side1_0003,PFP_Ph1_P0411_D01_S003_C2_az360_side1_0005,Ph1_P0031_D01_S001_C2_az080,0031_D01_S001,3,0411_D01_S003,MI1,False


In [108]:
for df in dfs:
#     df = dfs[1]

    sample_actual = df.iloc[0].sample_actual
    package_actual = df.iloc[0].package_actual

    alg_guesses = []
    for alg in df.algorithm.unique():
        df_piv = df[df.algorithm == alg].pivot_table(
            index='package_predicted', columns='descriptor_actual', values='distance', fill_value=np.nan,
            aggfunc=[np.mean, np.median, 'size', sum, min, 'std', np.var]#, ]
        )
        df_piv = df_piv.reset_index()

        df_rank = pd.DataFrame(columns=df_piv.columns)

        df_rank['index'] = df_piv['index']

        rank_ascending = ['mean', 'median', 'sum', 'std', 'var'] # 
        df_rank.loc[:, rank_ascending] = df_piv[rank_ascending].rank(method='dense', na_option='bottom', ascending=True)

        rank_descending = ['size']
        df_rank.loc[:, rank_descending] = df_piv[rank_descending].rank(method='dense', na_option='bottom', ascending=False)

        guess_rankings = dict()

        for desc_index in df_piv.columns.levels[1].to_list()[:-1]:
            df_rank.loc[:, ('rank_sum', desc_index)] = df_rank.loc[:, (rank_ascending + rank_descending, desc_index)].sum(axis=1)
            guesses = df_rank.loc[df_rank.loc[:, ('rank_sum', desc_index)].nsmallest(10).index, 'index'].to_list()

            for rank, guess in enumerate(guesses, start=1):
                if guess in guess_rankings.keys():
                    guess_rankings[guess].append(rank)
                else:
                    guess_rankings[guess] = [rank]

        top_guesses = [s[0] for s in sorted(guess_rankings.items(), key=lambda x: len(x[1]), reverse=True)][:10]
        alg_guesses.extend(top_guesses)
        #return [sample_actual, package_actual] + top_guesses
    print(package_actual in alg_guesses)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


KeyboardInterrupt: 

In [102]:
alg_guesses

{'MSER': ['0067_D01_S001',
  '0274_D01_S001',
  '0274_D01_S002',
  '0355_D01_S001',
  '0827_D01_S001',
  '0691_D01_S001',
  '0352_D01_S002',
  '0455_D01_S001',
  '0238_D01_S002',
  '0147_D01_S001'],
 'MI1': ['0031_D01_S001',
  '0059_D01_S001',
  '0751_D01_S001',
  '0737_D01_S001',
  '0155_D01_S001',
  '0069_D02_S001',
  '0320_D01_S001',
  '0793_D01_S001',
  '0505_D01_S001',
  '0758_D01_S001']}

In [50]:
dasd = df.descriptor_actual.max() // 10

df['package_predicted'].value_counts().reset_index(name="count").query(f"count > {dasd}")

,index,count
0,0274_D01_S002,39
1,0274_D01_S001,38
2,0067_D01_S001,35
3,0827_D01_S001,29
4,0691_D01_S001,24
5,0080_D01_S001,19
6,0355_D01_S001,19
7,0238_D01_S002,18
8,0238_D01_S003,17
9,0455_D01_S001,16


In [31]:
sorted(df_piv['size'].isnull().sum(axis=1).unique())

[43, 44, 46, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]

In [30]:
df_piv['size'].shape

(390, 62)

In [53]:
sample_actual

'Ph1_P0031_D01_S001_C2_az080'

In [77]:
df = dfs[2]

sample_actual = df.iloc[0].sample_actual
package_actual = df.iloc[0].package_actual
df[df.distance < 1].package_predicted.value_counts()

0383_D01_S001    1
Name: package_predicted, dtype: int64

In [83]:
df[df.algorithm == 'MI1'].descriptor_actual.max()

8